# Naver Blog Review Preprocessing 
* 한 개의 키워드에 대한 정제 코드
* 정제 후 Csv 저장까지 

## 사용 라이브러리 설치

[soynlp](https://github.com/lovit/soynlp)

In [1]:
# 한글 문장 분리기 
!pip install kss # korean sentence splitter

# 한글 맞춤법 검사기
!pip install requests # py-hanspell 의존 라이브러리
!pip install git+https://github.com/ssut/py-hanspell.git # py-hanspell
!pip install py-hangul-checker # py-hangul-checker

# 한글 띄어쓰기 교정기 
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
    
# 고유명사 추출에 용이한 토크나이저
!pip install soynlp

!pip install soyspacing

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-cn9i3dmq
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-cn9i3dmq
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-x6tr5ip9
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-x6tr5ip9
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


## 파일 경로 설정 

In [2]:
!mkdir -p ~/jungcheck/DataPreprocessing    
!pwd    
!cd ~/jungcheck    
!mkdir -p ./data/naver_review  

/home/aiffel-dj19/jungcheck/DataPreprocessing


In [3]:
# 현재 경로 확인 
!pwd

/home/aiffel-dj19/jungcheck/DataPreprocessing


아래 tree 명령어로 상위 폴더(jungcheck)내 파일 구성을 볼 수 있습니다.    

In [4]:
# jungcheck 폴더내 경로 확인 
!tree -L 3 -N ./data/naver_review  

./data/naver_review
├── 수통골
│   ├── 수통골 관광 대전.csv
│   ├── 수통골 여행 대전.csv
│   ├── 수통골 여행 후기.csv
│   ├── 수통골 관광 후기.csv
│   └── 수통골 여행 주말에.csv
├── 대청호
│   ├── 대청호반 관광 후기.csv
│   ├── 대청호반 관광 대전.csv
│   ├── 대청호반 여행 대전.csv
│   ├── 대청호반 여행 후기.csv
│   ├── 대청호반 관광 주말에.csv
│   └── 대청호반 여행 주말에.csv
├── 장태산
│   ├── 장태산휴양림 여행 후기.csv
│   ├── 장태산휴양림 관광 후기.csv
│   ├── 장태산휴양림 관광 대전.csv
│   ├── 장태산휴양림 여행 대전.csv
│   ├── 장태산휴양림 관광 주말에.csv
│   └── 장태산휴양림 여행 주말에.csv
├── 성심당
│   ├── 성심당 여행 후기.csv
│   ├── 성심당 여행 대전.csv
│   ├── 성심당 관광 대전.csv
│   ├── 성심당 관광 후기.csv
│   ├── 성심당 여행 주말에.csv
│   └── 성심당 관광 주말에.csv
├── 계족산
│   ├── 계족산황톳길 여행 대전.csv
│   ├── 계족산황톳길 관광 대전.csv
│   ├── 계족산황톳길 여행 후기.csv
│   ├── 계족산황톳길 관광 후기.csv
│   ├── 계족산황톳길 여행 주말에.csv
│   └── 계족산황톳길 관광 주말에.csv
├── 오월드
│   ├── 오월드 여행 대전.csv
│   ├── 오월드 관광 주말에.csv
│   └── 오월드 여행 주말에.csv
├── 동춘당
│   ├── 동춘당 관광 대전.csv
│   ├── 동춘당 관광 후기.csv
│   ├── 동춘당 여행 대전.csv
│   ├── 동춘당 여행 후기.csv
│   ├── 동춘당 여행 주말에.csv
│   └── 동춘당 관광 주말에.csv
├── 유성온천
│   ├── 유성온천 관광 후기.csv
│

## Data Load 

In [5]:
import os 
import pandas as pd 
import glob

In [6]:
data_path = os.path.join(os.getcwd(), 'data/naver_review/')
data_path

'/home/aiffel-dj19/jungcheck/DataPreprocessing/data/naver_review/'

In [7]:
folders = os.listdir(data_path)
len(folders)

17

In [8]:
keywords = ['장태산',
 '계족산',
 '동춘당',
 '으능정이문화의거리',
 '뿌리공원',
 '엑스포',
 '성심당',
 '수통골',
 '유성온천',
 '한밭수목원',
 '국립중앙과학관',
 '대청호',
 '대전문화예술단지',
 '시민천문대',
 '오월드',
 '한밭야구장',
 '대전둘레산길']

In [9]:
# keywords = list(set([file_name.split()[0] for file_name in files]))
print(len(keywords))
print(keywords)

17
['장태산', '계족산', '동춘당', '으능정이문화의거리', '뿌리공원', '엑스포', '성심당', '수통골', '유성온천', '한밭수목원', '국립중앙과학관', '대청호', '대전문화예술단지', '시민천문대', '오월드', '한밭야구장', '대전둘레산길']


In [10]:
keywords = {k:[] for k in keywords}
keywords

{'장태산': [],
 '계족산': [],
 '동춘당': [],
 '으능정이문화의거리': [],
 '뿌리공원': [],
 '엑스포': [],
 '성심당': [],
 '수통골': [],
 '유성온천': [],
 '한밭수목원': [],
 '국립중앙과학관': [],
 '대청호': [],
 '대전문화예술단지': [],
 '시민천문대': [],
 '오월드': [],
 '한밭야구장': [],
 '대전둘레산길': []}

dataframe만들때 keyword 동일하게 넣어줄것 => 하나의 데이터 프레임으로 만들어버리기 

In [11]:
# 대전관광 홈페이지 영문버전을 참고했으며(단, 긴 경우 줄임), 해당 리스트는 정제후 파일명으로 사용
# 나오지 않는 관광지는 네이버 검색 기준을 따름
keywords_ko = [file_name for file_name in keywords.keys()]
keywords_en = ['jangtae_mountain', 'gyejok_mountain', 'dongchundang', 'uineungjeongi_street', 'ppuri_park', 
               'expo_science_park', 'sungsimdang_bakery', 'water_barrel', 'yuseong_hotspring', 'hanbat_arboretum', 
               'science_museum','daecheong_lake', 'art_culture_complex', 'observatory', 
               'oworld_zoo', 'hanbat_baseball', 'dullesan_street']

keywords = {k:v for k, v in zip(keywords_ko, keywords_en)}
keywords

{'장태산': 'jangtae_mountain',
 '계족산': 'gyejok_mountain',
 '동춘당': 'dongchundang',
 '으능정이문화의거리': 'uineungjeongi_street',
 '뿌리공원': 'ppuri_park',
 '엑스포': 'expo_science_park',
 '성심당': 'sungsimdang_bakery',
 '수통골': 'water_barrel',
 '유성온천': 'yuseong_hotspring',
 '한밭수목원': 'hanbat_arboretum',
 '국립중앙과학관': 'science_museum',
 '대청호': 'daecheong_lake',
 '대전문화예술단지': 'art_culture_complex',
 '시민천문대': 'observatory',
 '오월드': 'oworld_zoo',
 '한밭야구장': 'hanbat_baseball',
 '대전둘레산길': 'dullesan_street'}

## keyword 폴더 하위의 파일들을 하나의 데이터프레임으로 합치기

In [12]:
# keyword = '오월드'
# keyword_en = 'oworld_zoo'

In [13]:
# keyword = '한밭야구장'
# keyword_en = 'hanbat_baseball'

In [14]:
keyword = '성심당'
keyword_en = keywords[keyword]
keyword_en

'sungsimdang_bakery'

In [15]:
files = os.listdir(data_path + '/' + keyword)
print(len(files))
print(files)

6
['성심당 여행 후기.csv', '성심당 여행 주말에.csv', '성심당 여행 대전.csv', '성심당 관광 주말에.csv', '성심당 관광 대전.csv', '성심당 관광 후기.csv']


In [16]:
# 파일명에 키워드가 포함된 것만 파일리스트에 포함 
files_list = []

for file_name in files:
#     print(file_name)
#     for keyword in keywords.keys():
    if keyword in file_name:
        files_list.append(file_name)
print(f'{files_list}')

['성심당 여행 후기.csv', '성심당 여행 주말에.csv', '성심당 여행 대전.csv', '성심당 관광 주말에.csv', '성심당 관광 대전.csv', '성심당 관광 후기.csv']


In [17]:
# root_path내의 dataframe을 합해주는 함수
def concatCsv(root_path:str, keys:dict, folder_name:str):
    '''
        root_path: 17개의 관광지 폴더를 가지고 있는 상위 폴더까지의 경로
        keys: 17개 관광지 키워드 사전(한국어: 영어)
        folder_name: 17개 관광지 중 1개의 관광지명으로 된 폴더명 
    '''
    data_list = []
    folder_path = os.path.join(root_path, folder_name)
    print(f'folder name: {folder_path}')

    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        print(f'file path: {file_path}')

        df = pd.read_csv(file_path, names=['date', 'description', 'text', 'name'], encoding='utf-8') 
        search = file.split('.')[0] # 파일명에서 검색어 추출 
        df['search'] = search
        df['keyword'] = folder_name 
        data_list.append(df)
        print(f'해당 파일의 데이터 개수: {len(df)}')
        print('-'*40)
        
    df = pd.concat(data_list, axis=0)
    
    return df

In [18]:
data_path

'/home/aiffel-dj19/jungcheck/DataPreprocessing/data/naver_review/'

In [19]:
data_df = concatCsv(data_path, keywords, keyword)

folder name: /home/aiffel-dj19/jungcheck/DataPreprocessing/data/naver_review/성심당
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/data/naver_review/성심당/성심당 여행 후기.csv
해당 파일의 데이터 개수: 2
----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/data/naver_review/성심당/성심당 여행 주말에.csv
해당 파일의 데이터 개수: 2
----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/data/naver_review/성심당/성심당 여행 대전.csv
해당 파일의 데이터 개수: 3
----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/data/naver_review/성심당/성심당 관광 주말에.csv
해당 파일의 데이터 개수: 10
----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/data/naver_review/성심당/성심당 관광 대전.csv
해당 파일의 데이터 개수: 3
----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/data/naver_review/성심당/성심당 관광 후기.csv
해당 파일의 데이터 개수: 1
----------------------------------------


In [20]:
data_df.head()

,date,description,text,name,search,keyword
0,21.05.21,마시고 성심당에서 사온 크레페랑 먹으면서 재밌는얘기하고 티비로 왓챠 연결해서 잼난거...,\n\n부띠끄에서 만족스럽게 후식까지 조지고 숙소로 이동~\n\n8번 출구에서 찍죽...,열심히 좀 살자!!,성심당 여행 후기,성심당
1,21.05.21,간만에 5일의 휴가가 생겨서 대전으로 1박2일 여행을 갔다왔다 엄마한테는 혼자 출발...,\n\n \n간만에 5일의 휴가가 생겨서 대전으로 1박2일 여행을 갔다왔다엄마한테는...,열심히 좀 살자!!,성심당 여행 후기,성심당
0,21.05.29,죽기 전에 가보고 싶은 국내 여행지 제주도 54. 추억이 있는 장소 대전 성심당 5...,\n\n1. 이름正恩(zhèngēn)2. 생일2003.01.023. 키170에서 -...,Hᴀᴘᴘʏ ᴅᴀɪʟʏ,성심당 여행 주말에,성심당
1,21.04.27,- 주말에 필 받아서 친구랑 성심당에서 빵먹고 오기. - 바람쐬고 싶으니 고속도로 ...,\n\n21년 4월 17일 : 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는...,그 녀석의 여행 이야기,성심당 여행 주말에,성심당
0,21.05.29,죽기 전에 가보고 싶은 국내 여행지 제주도 54. 추억이 있는 장소 대전 성심당 5...,\n\n1. 이름正恩(zhèngēn)2. 생일2003.01.023. 키170에서 -...,Hᴀᴘᴘʏ ᴅᴀɪʟʏ,성심당 여행 대전,성심당


In [21]:
len(data_df)

21

## Data 확인

1. 중복 데이터 확인 
2. 중복 데이터 제거 
3. 기간 19~21년도로 필터링

In [22]:
# 블로그명 겹치는거 없는지 확인
duplicated_name = len(data_df)-len(list(set(data_df['name'])))
print(f'불러온 데이터셋은 {duplicated_name}개의 블로그명이 겹칩니다.')

불러온 데이터셋은 2개의 블로그명이 겹칩니다.


In [23]:
# 블로그 본문 겹치는거 없는지 확인
duplicated_txt = len(data_df)-len(list(set(data_df['text'])))
print(f'불러온 데이터셋은 {duplicated_txt}개의 블로그명이 겹칩니다.')

불러온 데이터셋은 1개의 블로그명이 겹칩니다.


In [24]:
# 블로그 아예 동일한 데이터 없는지 확인
duplicated_df = data_df.duplicated(['date', 'text', 'name']).sum()
print(f'불러온 데이터셋은 {duplicated_df}개의 블로그 내용이 겹칩니다.')

불러온 데이터셋은 1개의 블로그 내용이 겹칩니다.


In [25]:
# 중복 제거 
data_df = data_df.drop_duplicates(['date', 'description', 'name'])
len(data_df)

21

In [26]:
# 전체 option 확인
# pd.describe_option()

In [27]:
# get pandas option 
print(pd.get_option('display.max_rows'))
print(pd.get_option('display.max_columns'))
print(pd.get_option('display.expand_frame_repr'))

60
20
True


In [28]:
# # set pandas option 
# pd.set_option('display.max_rows', None) # 전체 행 보기
# pd.set_option('display.max_columns', None) # 전체 열 보기 
# pd.set_option('display.expand_frame_repr', False)

In [29]:
# date에서 '.' 제거후 str에서 int로 변환 
for i in range(len(data_df['date'])):
    data_df['date'].iloc[i] = int(str(data_df['date'].iloc[i]).replace('.',''))

In [30]:
# # date에서 str에서 int로 변환 
# data_df['date'] = data_df['date'].apply(lambda row: int(row))
# data_df.head()

In [31]:
type(data_df['date'].iloc[0])

int

In [32]:
date_list = list(set(data_df['date']))
date_list

[210529,
 191012,
 191018,
 190923,
 190922,
 210425,
 190702,
 210415,
 210320,
 190321,
 181109,
 201014,
 210521,
 210427,
 190525,
 210526,
 190623]

In [33]:
data_df = data_df[data_df['date'] >= 190101]

In [34]:
data_df = data_df[data_df['date'] <= 210501]

In [35]:
date_list = [i for i in date_list if i not in list(set(data_df['date']))]
print(f'필터링된 날짜는: {date_list}')

필터링된 날짜는: [210529, 181109, 210521, 210526]


In [36]:
data_df['date'].min()

190321

In [37]:
data_df['date'].max()

210427

In [38]:
len(data_df)

14

In [39]:
# list(set(data_df['text']))

In [40]:
data_df.head()

,date,description,text,name,search,keyword
1,210427,- 주말에 필 받아서 친구랑 성심당에서 빵먹고 오기. - 바람쐬고 싶으니 고속도로 ...,\n\n21년 4월 17일 : 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는...,그 녀석의 여행 이야기,성심당 여행 주말에,성심당
0,191018,이제 주말에 여유 생길 테니 자주 볼 수 있으면 좋겠다.^^ 2019대전 : 20개...,\n\n2019대전 : 20개월 아기와 함께 당일치기 여행 (자차+KTX)첫차를 산...,봄날,성심당 관광 주말에,성심당
1,191012,- 2019년 10월 12일 토요일 주말에 신랑이 같이 일하는 형님들과 배낚시를 간...,\n\n-2019년 10월 12일 토요일 주말에 신랑이 같이 일하는 형님들과 배낚시...,; ] 양 뚱의 BLOG,성심당 관광 주말에,성심당
2,190923,음식을 성심당 것 차 이들이 경기권으로 제가 주말에 들어가서 많아 다 법령도 열심히...,\n\n안녕하세요날이 갈수록 서늘해지는 가을로 접어들었습니다. 활동하기 알맞은날씨와...,부산현수막,성심당 관광 주말에,성심당
3,190923,주말에 태풍소식이 ㅠㅠ 이전 태풍때는 바람만 불기도 했었고 그 전 태풍때 서울엔 흐...,"\n\n대전온천여행이 되버린 주말, 유성호텔 수영장 + 대욕장에서 놀고왔어요저희가족...",서남매와 제주살이중_♥,성심당 관광 주말에,성심당


## 문장 분리 

토크나이저를 활용해 'text'에 해당하는 네이버 블로그 본문에 대한 문장 분리를 진행해줍니다. 이러한 문장 분리 관련한 자연어 처리를 Sentence boundary disambiguation(SBD)라고 합니다. 해당 키워드로 검색하면 관련한 자료들이 많이 나옵니다.
* [kss(korean sentence splitter)](http://docs.likejazz.com/kss/) 활용 => 한글 SBD 해결에 유용
* [spaCy](https://www.facebook.com/groups/TensorFlowKR/permalink/1025310851143306/) => 영어 SBD 해결에 유용

In [41]:
import kss 

In [42]:
# 문장에 마침표 없는 경우 어떻게 문장 분리 수행하는지 확인
test = ['과연 어떻게 할지 궁금합니다 이거는 마침표없는 문장인데 어떻게 분리해낼까 할 수 있을까나 걱정됩니다']
test = list(map(kss.split_sentences, test)) # 문장으로 split
test

[['과연 어떻게 할지 궁금합니다', '이거는 마침표없는 문장인데 어떻게 분리해낼까 할 수 있을까나 걱정됩니다']]

In [43]:
# 문장에 마침표 없는 경우 어떻게 문장 분리 수행하는지 확인
test = ['과연 어떻게 할지 궁금하다 이거는 마침표없는 문장인데 어떻게 분리해낼까 할 수 있을까나 걱정됩니다']
test = list(map(kss.split_sentences, test)) # 문장으로 split
test

[['과연 어떻게 할지 궁금하다 이거는 마침표없는 문장인데 어떻게 분리해낼까 할 수 있을까나 걱정됩니다']]

In [44]:
data_df['text'] = data_df['text'].map(kss.split_sentences) # dataframe 문장으로 split

In [45]:
# 문장별로 확인 
test_list = data_df['text'].iloc[0]
print(len(test_list)) # 문장 개수

for i in range(len(test_list)): # 내용 확인
    print(f'{i}번째 문장: {test_list[i]}')

88
0번째 문장: 21년 4월 17일 : 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다 좀 벽이 있고 특별히 어려운 관계는 아니지만 그렇다고 일부러 연락해서 만나고 그러기엔 엄청 어색하다고..우리 집안이 좀 이상한건지 대부분의 사촌동생들과 아주 친하게 지낸다.
1번째 문장: 자주 연락하고 같이 식사, 술 마시고 거리낌 없이 집에서 자고가기도 하고..아...
2번째 문장: 참고로 아버지 집안의 형제가 7남매 + 어머니 집안은 8남매 대부분 2명의 자녀라 내 사촌들의 수는 엄청나다..
3번째 문장: 크큽..사회 초년생의 사촌들은 아무래도 내가 삼촌같아서 잘 어울리긴 힘들고나이가 엇 비슷한 사촌들과 좀 더 친하긴 하다.
4번째 문장: 그 사촌 동생들 중 JH이라는 사촌이 있는데 오랜기간 공부하고 연구해서 박사가 되더니최근까지 서울에 대학에서 연구를 하다가 올해 대전에 있는 연구소로 취업을 하였다.
5번째 문장: 회사에 있는동안 오피스텔이나 기숙사에서 지낼걸 생각했더니 이래저래 해서 대전 둔산동에 집을 매입했고 내부 인테리어 마쳤다고 했다.
6번째 문장: 집들이도 할겸, 동생 취업도 축하도 할겸마침 날씨는 좋은데 주말에 몸이 근질근질 하기도 했고1년정도의 장기간 코로나로 집에 더 뒹굴면 진짜 우울증 심하게 올것 같은 느낌이였다.
7번째 문장: 그렇게 토요일 오후 차를 끌고 대전 둔산동 사촌집으로 갔다.
8번째 문장: 야탑에 있는 회사에서 출발하였는데 2시간 좀 넘게 걸렸다.고속도로는 막히지 않았는데 혼자가려니 엄청 지루했다.
9번째 문장: 둔산동 동생집 주차장에 차를 세우고 간단히 집 구경을 하고 대부분 만족스러웠지만정리 빈약한 부분 오지랍 하면서 갈구기도 하고 식사를 하러 나왔다.
10번째 문장: 출발 전부터 오늘은 이상하게 양꼬치가 먹고 싶었다.
11번째 문장: 지글지글한 양꼬치에 맥주가 너무나 땡기던날..사실 난 대전과 연관되는게 없어 주기적인 방문은 없다.
12번째 문장: - 주말에 필 받아서 친구랑 성심당에서 빵먹고 오기.- 바람쐬고 싶으

kss는 마침표가 없으면 한 문으로 인식하는 문제점이 있습니다. 그래서 normalize를 시도해서 한번 문장 정제를 거치고 split해봤습니다. 해도 안될시에는 [hanspell 라이브러리](https://ebbnflow.tistory.com/246)를 사용해볼 예정입니다.

## 문단 정규표현식

* [정규표현식 설명 블로그](https://blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221347960543)
* [정규표현식 모음](https://velog.io/@ash3767/python-%EC%A0%95%EA%B7%9C%EC%8B%9D)

In [46]:
import re

In [47]:
# def senPreprocessing(sen):
#     sen = re.sub(pattern='^[^((http(s?))\:\/\/)]([0-9a-zA-Z\-]+\.)+[a-zA-Z]{2,6}(\:[0-9]+)?(\/\S*)?$', repl=' ', string=sen) 
#     sen = re.sub(pattern='^([\w-]+(?:\.[\w-]+)*)@((?:[\w-]+\.)*\w[\w-]{0,66})\.([a-z]{2,6}(?:\.[a-z]{2})?)$', repl=' ', string=sen).strip() # 이메일 삭제
#     sen = re.sub(pattern='\([^)]*\)', repl='', string=sen) 
#     sen = re.sub(pattern='\<[^>]*\)', repl='', string=sen) 
#     sen = re.sub(pattern='\[[^]]*\)', repl='', string=sen) 
#     sen = re.sub(pattern='[-=+,#/\:^$@*\"※~&%ㆍ』\\‘|\(\)\[\]\<\>`\'…》]', repl='', string=sen)
#     sen = re.sub(pattern='[\n\xa0]', repl='', string=sen) 
#     sen = re.sub(pattern='[ㄱ-ㅎㅏ-ㅣ]+', repl='', string=sen) 
#     sen = re.sub(pattern='[?!~.]?', repl='', string=sen) # 마침표 역할하는 기호 제거  
#     sen = re.sub(pattern='\s+', repl=' ', string=sen).strip() # 띄어쓰기 1개씩만 
#     sen = re.sub(pattern='/^[가-힣\s]+$/', repl=' ', string=sen).strip()
#     return sen

In [48]:
def senPreprocessing(sen):
    print(sen)
    pattern = '\S*@\S*\s?' 
    sen = re.sub(pattern=pattern, repl='', string=sen)
    print("E-mail제거 : " , sen)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    sen = re.sub(pattern=pattern, repl='', string=sen)
    print("URL 제거 : ", sen)
    pattern = '(\d{3}).*(\d{3}).*(\d{4})+'
    sen = re.sub(pattern=pattern, repl='', string=sen)
    print("번호 제거 : ", sen)
    pattern = '[|ㄱ-ㅎ|ㅏ-ㅣ|a-z|A-Z]+'  
    sen = re.sub(pattern=pattern, repl='', string=sen)
    print("한글 자음 모음 영문 제거 : ", sen)
    pattern = '<[^>]*>'        
    sen = re.sub(pattern=pattern, repl='', string=sen)
    print("태그 제거 : " , sen)
    pattern = '\([^)]*\)'
    sen = re.sub(pattern=pattern, repl='', string=sen)
    print("괄호와 괄호안 글자 제거 :  " , sen)
    pattern = '[^\w\s]'   
    sen = re.sub(pattern=pattern, repl='', string=sen)
    print("특수기호 제거 : ", sen)
    sen = sen.strip()
    print("양 끝 공백 제거 : ", sen)
    sen = " ".join(sen.split())
    print("중간에 공백은 1개만 : ", sen)
    return sen

In [49]:
senPreprocessing('dk  010-2232-1230    test@naver.com  http://naver.com ㅋ<ㅋ>[ㅋ]ㅋ(ㅋ)ㅋ   아 ')

dk  010-2232-1230    test@naver.com  http://naver.com ㅋ<ㅋ>[ㅋ]ㅋ(ㅋ)ㅋ   아 
E-mail제거 :  dk  010-2232-1230     http://naver.com ㅋ<ㅋ>[ㅋ]ㅋ(ㅋ)ㅋ   아 
URL 제거 :  dk  010-2232-1230      ㅋ<ㅋ>[ㅋ]ㅋ(ㅋ)ㅋ   아 
번호 제거 :  dk        ㅋ<ㅋ>[ㅋ]ㅋ(ㅋ)ㅋ   아 
한글 자음 모음 영문 제거 :          <>[]()   아 
태그 제거 :          []()   아 
괄호와 괄호안 글자 제거 :           []   아 
특수기호 제거 :             아 
양 끝 공백 제거 :  아
중간에 공백은 1개만 :  아


'아'

이후에 맞춤법 교정에서 띄어쓰기를 교정할때 문장이 애매하게 띄어쓰기 되어있는 것보다, 아예 이어붙어진 경우가 교정이 더 잘 됩니다.

In [50]:
from tqdm import tqdm

In [51]:
for i in range(len(data_df)):
    length = len(data_df['text'].iloc[i])
    print(f'{i}번째 데이터: {length}개의 문장')

0번째 데이터: 88개의 문장
1번째 데이터: 10개의 문장
2번째 데이터: 25개의 문장
3번째 데이터: 17개의 문장
4번째 데이터: 42개의 문장
5번째 데이터: 5개의 문장
6번째 데이터: 14개의 문장
7번째 데이터: 20개의 문장
8번째 데이터: 51개의 문장
9번째 데이터: 1개의 문장
10번째 데이터: 46개의 문장
11번째 데이터: 21개의 문장
12번째 데이터: 2개의 문장
13번째 데이터: 79개의 문장


In [52]:
# dataframe 각 행내 리스트로 된 문장 단위 글에 정규표현식 적용
for i in tqdm(range(len(data_df))):
    data_list = data_df['text'].iloc[i]
    data_df['text'].iloc[i] = list(map(senPreprocessing, data_list)) 

 14%|█▍        | 2/14 [00:00<00:00, 17.39it/s]

21년 4월 17일 : 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다 좀 벽이 있고 특별히 어려운 관계는 아니지만 그렇다고 일부러 연락해서 만나고 그러기엔 엄청 어색하다고..우리 집안이 좀 이상한건지 대부분의 사촌동생들과 아주 친하게 지낸다.
E-mail제거 :  21년 4월 17일 : 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다 좀 벽이 있고 특별히 어려운 관계는 아니지만 그렇다고 일부러 연락해서 만나고 그러기엔 엄청 어색하다고..우리 집안이 좀 이상한건지 대부분의 사촌동생들과 아주 친하게 지낸다.
URL 제거 :  21년 4월 17일 : 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다 좀 벽이 있고 특별히 어려운 관계는 아니지만 그렇다고 일부러 연락해서 만나고 그러기엔 엄청 어색하다고..우리 집안이 좀 이상한건지 대부분의 사촌동생들과 아주 친하게 지낸다.
번호 제거 :  21년 4월 17일 : 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다 좀 벽이 있고 특별히 어려운 관계는 아니지만 그렇다고 일부러 연락해서 만나고 그러기엔 엄청 어색하다고..우리 집안이 좀 이상한건지 대부분의 사촌동생들과 아주 친하게 지낸다.
한글 자음 모음 영문 제거 :  21년 4월 17일 : 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다 좀 벽이 있고 특별히 어려운 관계는 아니지만 그렇다고 일부러 연락해서 만나고 그러기엔 엄청 어색하다고..우리 집안이 좀 이상한건지 대부분의 사촌동생들과 아주 친하게 지낸다.
태그 제거 :  21년 4월 17일 : 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다 좀 벽이 있고 특별히 어려운 관계는 아니지만 그렇다고 일부러 연락해서 만나고 그러기엔 엄청 어색하다고..우리 집안이 좀 이상한건지 대부분의 사촌동생들과 아주 친하게 지낸다.
괄호와 괄호안 글자 제거 :   21년 4월 17일 : 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다

한글 자음 모음 영문 제거 :  건축개요를 살펴보면 매우 넓은부지를 확인할수 있겠고, 지하1층에서 지상 4층 규모로 15개동에 201세대가 입주하게 됩니다.
태그 제거 :  건축개요를 살펴보면 매우 넓은부지를 확인할수 있겠고, 지하1층에서 지상 4층 규모로 15개동에 201세대가 입주하게 됩니다.
괄호와 괄호안 글자 제거 :   건축개요를 살펴보면 매우 넓은부지를 확인할수 있겠고, 지하1층에서 지상 4층 규모로 15개동에 201세대가 입주하게 됩니다.
특수기호 제거 :  건축개요를 살펴보면 매우 넓은부지를 확인할수 있겠고 지하1층에서 지상 4층 규모로 15개동에 201세대가 입주하게 됩니다
양 끝 공백 제거 :  건축개요를 살펴보면 매우 넓은부지를 확인할수 있겠고 지하1층에서 지상 4층 규모로 15개동에 201세대가 입주하게 됩니다
중간에 공백은 1개만 :  건축개요를 살펴보면 매우 넓은부지를 확인할수 있겠고 지하1층에서 지상 4층 규모로 15개동에 201세대가 입주하게 됩니다
공동 주택과 부대 복리 시설로들어오게 됩니다지하주차장도 있고, 306대의 자동차를 주차할 수 있었습니다.
E-mail제거 :  공동 주택과 부대 복리 시설로들어오게 됩니다지하주차장도 있고, 306대의 자동차를 주차할 수 있었습니다.
URL 제거 :  공동 주택과 부대 복리 시설로들어오게 됩니다지하주차장도 있고, 306대의 자동차를 주차할 수 있었습니다.
번호 제거 :  공동 주택과 부대 복리 시설로들어오게 됩니다지하주차장도 있고, 306대의 자동차를 주차할 수 있었습니다.
한글 자음 모음 영문 제거 :  공동 주택과 부대 복리 시설로들어오게 됩니다지하주차장도 있고, 306대의 자동차를 주차할 수 있었습니다.
태그 제거 :  공동 주택과 부대 복리 시설로들어오게 됩니다지하주차장도 있고, 306대의 자동차를 주차할 수 있었습니다.
괄호와 괄호안 글자 제거 :   공동 주택과 부대 복리 시설로들어오게 됩니다지하주차장도 있고, 306대의 자동차를 주차할 수 있었습니다.
특수기호 제거 :  공동

괄호와 괄호안 글자 제거 :   거기는 주변에 성심당 말고 아무것도 없기도 하고..온천이 따로 없어요.
특수기호 제거 :  거기는 주변에 성심당 말고 아무것도 없기도 하고온천이 따로 없어요
양 끝 공백 제거 :  거기는 주변에 성심당 말고 아무것도 없기도 하고온천이 따로 없어요
중간에 공백은 1개만 :  거기는 주변에 성심당 말고 아무것도 없기도 하고온천이 따로 없어요
성인만 갈 수 있는 사우나 비슷한게 있는데아이들은 갈수가 없어서 유성호텔로 했죠.
E-mail제거 :  성인만 갈 수 있는 사우나 비슷한게 있는데아이들은 갈수가 없어서 유성호텔로 했죠.
URL 제거 :  성인만 갈 수 있는 사우나 비슷한게 있는데아이들은 갈수가 없어서 유성호텔로 했죠.
번호 제거 :  성인만 갈 수 있는 사우나 비슷한게 있는데아이들은 갈수가 없어서 유성호텔로 했죠.
한글 자음 모음 영문 제거 :  성인만 갈 수 있는 사우나 비슷한게 있는데아이들은 갈수가 없어서 유성호텔로 했죠.
태그 제거 :  성인만 갈 수 있는 사우나 비슷한게 있는데아이들은 갈수가 없어서 유성호텔로 했죠.
괄호와 괄호안 글자 제거 :   성인만 갈 수 있는 사우나 비슷한게 있는데아이들은 갈수가 없어서 유성호텔로 했죠.
특수기호 제거 :  성인만 갈 수 있는 사우나 비슷한게 있는데아이들은 갈수가 없어서 유성호텔로 했죠
양 끝 공백 제거 :  성인만 갈 수 있는 사우나 비슷한게 있는데아이들은 갈수가 없어서 유성호텔로 했죠
중간에 공백은 1개만 :  성인만 갈 수 있는 사우나 비슷한게 있는데아이들은 갈수가 없어서 유성호텔로 했죠
저번에 왔을때 온천을 갔었는데, 여탕은 동네 목욕탕 같은 수준이었지만 물이 좋았고.. 남탕은 노천탕도 있다고 해서 이번에도 여기로 했어요.
E-mail제거 :  저번에 왔을때 온천을 갔었는데, 여탕은 동네 목욕탕 같은 수준이었지만 물이 좋았고.. 남탕은 노천탕도 있다고 해서 이번에도 여기로 했어요.
URL 제거 :  저번에 왔을때 온천을 갔었는데, 여탕은 동네 목욕탕 같은 수준이었지만 물이 

중간에 공백은 1개만 :  고민되네요
태풍때문에 야구대신 대전온천여행 즐기고온 웃픈후기였습니다.
E-mail제거 :  태풍때문에 야구대신 대전온천여행 즐기고온 웃픈후기였습니다.
URL 제거 :  태풍때문에 야구대신 대전온천여행 즐기고온 웃픈후기였습니다.
번호 제거 :  태풍때문에 야구대신 대전온천여행 즐기고온 웃픈후기였습니다.
한글 자음 모음 영문 제거 :  태풍때문에 야구대신 대전온천여행 즐기고온 웃픈후기였습니다.
태그 제거 :  태풍때문에 야구대신 대전온천여행 즐기고온 웃픈후기였습니다.
괄호와 괄호안 글자 제거 :   태풍때문에 야구대신 대전온천여행 즐기고온 웃픈후기였습니다.
특수기호 제거 :  태풍때문에 야구대신 대전온천여행 즐기고온 웃픈후기였습니다
양 끝 공백 제거 :  태풍때문에 야구대신 대전온천여행 즐기고온 웃픈후기였습니다
중간에 공백은 1개만 :  태풍때문에 야구대신 대전온천여행 즐기고온 웃픈후기였습니다
점심먹고 좀 쉬다 다시 출발해서 전북 익산 진입길가에 바람개비가 이뻐서 한컷.또 달려서 논산 강경쯤 도착논산훈련소 앞에 한번 가볼까 했는데거리가 꽤 멀어서 패스 ㅋㅋ 이동네는 젓갈이 유명한가보네부여에 진입백제의 수도답게 회전교차로 중간에 떡하니 백제 금동대향로가 있네여기서 부여 박물관에 한번 가보고 싶었는데 시간이 늦어져서 그냥 지나왔다백제보 와서 잠깐 휴식멈춘김에 전망대에 한번 올라가볼까안에 들어오니 에어컨이 안돌고 있어서 엄청나게 더움망원경으로 여기저기를 둘러볼수 있다백제보백제보는 계백장군이 탄 말의 안장을 형상화 했다고 한다금강 상류쪽을 바라본다오늘도 날씨가 좋아서 참 다행이네멈추면 좀 덥고 달리면 차가운바람때문에 좀 쌀쌀한 느낌 세종시 도착네비가 안내하는대로 가다보니 길이 갑자기 사라졌다 ㅋㅋ
E-mail제거 :  점심먹고 좀 쉬다 다시 출발해서 전북 익산 진입길가에 바람개비가 이뻐서 한컷.또 달려서 논산 강경쯤 도착논산훈련소 앞에 한번 가볼까 했는데거리가 꽤 멀어서 패스 ㅋㅋ 이동네는 젓갈이 유명한가보네부여에 진입백제의 수도답게 회전교차로

...

한글 자음 모음 영문 제거 :  ◇“동네빵집 살길…세분화된 메뉴 개발”= 주 대표는 앞으로 동네빵집들이 나아가야 할 방향에 대해 ‘세분화된 메뉴’에서 찾아야 한다고 강조한다.
태그 제거 :  ◇“동네빵집 살길…세분화된 메뉴 개발”= 주 대표는 앞으로 동네빵집들이 나아가야 할 방향에 대해 ‘세분화된 메뉴’에서 찾아야 한다고 강조한다.
괄호와 괄호안 글자 제거 :   ◇“동네빵집 살길…세분화된 메뉴 개발”= 주 대표는 앞으로 동네빵집들이 나아가야 할 방향에 대해 ‘세분화된 메뉴’에서 찾아야 한다고 강조한다.
특수기호 제거 :  동네빵집 살길세분화된 메뉴 개발 주 대표는 앞으로 동네빵집들이 나아가야 할 방향에 대해 세분화된 메뉴에서 찾아야 한다고 강조한다
양 끝 공백 제거 :  동네빵집 살길세분화된 메뉴 개발 주 대표는 앞으로 동네빵집들이 나아가야 할 방향에 대해 세분화된 메뉴에서 찾아야 한다고 강조한다
중간에 공백은 1개만 :  동네빵집 살길세분화된 메뉴 개발 주 대표는 앞으로 동네빵집들이 나아가야 할 방향에 대해 세분화된 메뉴에서 찾아야 한다고 강조한다
과거에도 그랬고, 지금도 많은 빵집들의 메뉴는 백화점식 형태가 주류를 이룬다는 것.
E-mail제거 :  과거에도 그랬고, 지금도 많은 빵집들의 메뉴는 백화점식 형태가 주류를 이룬다는 것.
URL 제거 :  과거에도 그랬고, 지금도 많은 빵집들의 메뉴는 백화점식 형태가 주류를 이룬다는 것.
번호 제거 :  과거에도 그랬고, 지금도 많은 빵집들의 메뉴는 백화점식 형태가 주류를 이룬다는 것.
한글 자음 모음 영문 제거 :  과거에도 그랬고, 지금도 많은 빵집들의 메뉴는 백화점식 형태가 주류를 이룬다는 것.
태그 제거 :  과거에도 그랬고, 지금도 많은 빵집들의 메뉴는 백화점식 형태가 주류를 이룬다는 것.
괄호와 괄호안 글자 제거 :   과거에도 그랬고, 지금도 많은 빵집들의 메뉴는 백화점식 형태가 주류를 이룬다는 것.
특수기호 제거 :  과거에도 그랬고 지금도 많은 빵집들의 메뉴는 백화점식 형태가 주류를 이룬다는 것

 86%|████████▌ | 12/14 [00:00<00:00, 32.33it/s]

특수기호 제거 :  대사동 친구가 여기서 엄마 갖다 주라고 마들렌도 사줘서 한동안 식으로 잘 드심  마들렌이 크고 맛있음요  간만에 대학생 시절로 돌아간듯 성심당에서 빵먹고 빙수 먹고 수다 떨고 빵구경 빵샤핑 실컷했떤 즐거운 하루였습니다 
양 끝 공백 제거 :  대사동 친구가 여기서 엄마 갖다 주라고 마들렌도 사줘서 한동안 식으로 잘 드심  마들렌이 크고 맛있음요  간만에 대학생 시절로 돌아간듯 성심당에서 빵먹고 빙수 먹고 수다 떨고 빵구경 빵샤핑 실컷했떤 즐거운 하루였습니다
중간에 공백은 1개만 :  대사동 친구가 여기서 엄마 갖다 주라고 마들렌도 사줘서 한동안 식으로 잘 드심 마들렌이 크고 맛있음요 간만에 대학생 시절로 돌아간듯 성심당에서 빵먹고 빙수 먹고 수다 떨고 빵구경 빵샤핑 실컷했떤 즐거운 하루였습니다
맛있게 잘 먹었습니다.
E-mail제거 :  맛있게 잘 먹었습니다.
URL 제거 :  맛있게 잘 먹었습니다.
번호 제거 :  맛있게 잘 먹었습니다.
한글 자음 모음 영문 제거 :  맛있게 잘 먹었습니다.
태그 제거 :  맛있게 잘 먹었습니다.
괄호와 괄호안 글자 제거 :   맛있게 잘 먹었습니다.
특수기호 제거 :  맛있게 잘 먹었습니다
양 끝 공백 제거 :  맛있게 잘 먹었습니다
중간에 공백은 1개만 :  맛있게 잘 먹었습니다
세종시에서 아직 미개발인 상권이 대평동 고속터미널 상권입니다고속터미널 주변은 코스트코 외에는 미개발이고항아리 상권은 완성되고 있지만 ,,아직활성화가 안되고 있습니다세종터미널역+BRT+고속터미널3개의 트리풀 역세권이고..여기에 발산리 KTX까지 확정이 되면 쿼러불 역세권이 됩니다앞으로 세종시에서 가장 변화가 많이 예상이 되고,,상권의 확장성을 가지고 있습니다﻿고속터미널 상권에 개발 속도가 붙으려면역시 고속터미널을 많이 이용하여 유동인구가 많아야 하는데유동인구가 많으려면 몇가지 조건이 필요합니다첫째는, 서울세종 고속도로 완공둘째는 세종대전 광역전철세째는 발산리 주변 배후 주거지 개발(개발행위 제한구역 지정)아마도 이런 조건이 갖추

100%|██████████| 14/14 [00:00<00:00, 28.77it/s]

태그 제거 :  허브와 음성 마늘빵으로 유명한 류재은 베이커리 지점이 헤이리 예술마을에도 있으니 중간에 출출하시면 방문해 구매해 드시는 것도 괜찮아요﻿만원이하주식 
   ,   ,             .          ,           .  .                 .    2,500   -      ,    ,      .  17:00)    -,  -,  ,      .       ,       .  ( . 1709)          .     ( )              .    ,      119-17 -, -, ,        .   ,              .   ,  ⓒ             ,    ~     .                    ,            .       ,         .            14.     .﻿만원이하주식 
 6천여평의 초원에는 제1목장에서 3목장까지 있고 시원한 계곡이 흐르는 공기 맑고 경치 좋은 곳이었어요우리가 주문한 메뉴는 러블리세트로 제철 활어회와 해산물, 우럭찜, 매운탕이 나오는 코스로 구성되었어요 실전에 들어가기전 간단한 교육과 장비들을 점검하고 짝을 지어 카약을 바다에 옮겼는데요  이번엔 제대로 돌아볼 수 있었어요점심을 든든하게 먹고 짧은 휴식을 취한뒤 카약에 나섰어요 무우와 감자를 넣어 매콤달콤하게 맛볼 수 있었어요 이왕 먹는거 고루 맛보고 싶은 마음에서였는데요 이 곳의 메인은 회지만, 유명하기론 우럭찜도 엄지척인데요 문의:﻿만원이하주식 
녹슬고, 닳고, 부서지고있어 하 하하하 하하하 오노, 배우 한석규가 세운 초원 사진관의 이름 ~영화에서 나온 너무 많은 로맨스 영화가 우리 마음을 느꼈다고 생각합니다. 에피소드가 진행되는 동안 솜을 펴고 소금을 뿌려 눈처럼 장식합니다.주머니는 모아서 김치로 사용되었습니다. 군산에 갈만한 곳으로 이곳을 선택한 이유그것은 하나가되는 곳입니다. 대전 도청사를 근현대 사진관으로 활용 한 듯 아픈 기억이있는 곳이지만 관광 자산으로 다시 태어

In [53]:
# 문장별로 확인 
test_list = data_df['text'].iloc[0]
print(len(test_list)) # 문장 개수

for i in range(len(test_list)): # 5문장 정도만 내용 확인
    print(f'{i}번째 문장: {test_list[i]}')
    if i == 5: 
        print('...')
        break

88
0번째 문장: 21년 4월 17일 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다 좀 벽이 있고 특별히 어려운 관계는 아니지만 그렇다고 일부러 연락해서 만나고 그러기엔 엄청 어색하다고우리 집안이 좀 이상한건지 대부분의 사촌동생들과 아주 친하게 지낸다
1번째 문장: 자주 연락하고 같이 식사 술 마시고 거리낌 없이 집에서 자고가기도 하고아
2번째 문장: 참고로 아버지 집안의 형제가 7남매 어머니 집안은 8남매 대부분 2명의 자녀라 내 사촌들의 수는 엄청나다
3번째 문장: 크큽사회 초년생의 사촌들은 아무래도 내가 삼촌같아서 잘 어울리긴 힘들고나이가 엇 비슷한 사촌들과 좀 더 친하긴 하다
4번째 문장: 그 사촌 동생들 중 이라는 사촌이 있는데 오랜기간 공부하고 연구해서 박사가 되더니최근까지 서울에 대학에서 연구를 하다가 올해 대전에 있는 연구소로 취업을 하였다
5번째 문장: 회사에 있는동안 오피스텔이나 기숙사에서 지낼걸 생각했더니 이래저래 해서 대전 둔산동에 집을 매입했고 내부 인테리어 마쳤다고 했다
...


In [54]:
# 맺음말?에 해당하는 경우 .를 붙여줍니다.
sent ='그랬습니다' 
if sent[-1] in ['다', '요', '죠', '오', '자', '라', '가', '네']:
    print(sent + '.')

그랬습니다.


In [55]:
def dotAdd(sen):
    for end in ['다', '요', '죠', '오', '자', '라', '가', '네', '나', '까']:
        if sen.endswith(end):
            sen = sen + '.'
    return sen

In [56]:
list(map(dotAdd, data_df['text'].iloc[i]))[:5]

['점심먹고 좀 쉬다 다시 출발해서 전북 익산 진입길가에 바람개비가 이뻐서 한컷또 달려서 논산 강경쯤 도착논산훈련소 앞에 한번 가볼까 했는데거리가 꽤 멀어서 패스 이동네는 젓갈이 유명한가보네부여에 진입백제의 수도답게 회전교차로 중간에 떡하니 백제 금동대향로가 있네여기서 부여 박물관에 한번 가보고 싶었는데 시간이 늦어져서 그냥 지나왔다백제보 와서 잠깐 휴식멈춘김에 전망대에 한번 올라가볼까안에 들어오니 에어컨이 안돌고 있어서 엄청나게 더움망원경으로 여기저기를 둘러볼수 있다백제보백제보는 계백장군이 탄 말의 안장을 형상화 했다고 한다금강 상류쪽을 바라본다오늘도 날씨가 좋아서 참 다행이네멈추면 좀 덥고 달리면 차가운바람때문에 좀 쌀쌀한 느낌 세종시 도착네비가 안내하는대로 가다보니 길이 갑자기 사라졌다.',
 '여긴 어디 난 누구인가.',
 '오늘 마지막 종착지인 대청호에 도착 대청호에 반사된 하늘이 너무 이쁘다가슴이 탁 트이는구만파노라마 한컷 찍어줌금강 최상류 대청댐일부러 강을 따라 계속 움직였는데옛날 강주변을 따라 항상 도시가 형성되어왔고그래서 그길을 따라오는것이 볼게 많았던거 같다 해가 지기전에 대구 시내에 도착했다멀리 한빛탑이 보이네대전하면 제일 먼저 떠오르는게 이 탑 아닐까 꿈돌이도 아직 있네무슨행사를 준비하는지 광장이 정신이 없다주말에 태풍온다.',
 '했는데',
 '대전 왔으니 성심당 성심당 본점은 좀 멀리 있어서호텔 근처에 있는 대전 컨벤션 센터점에 왔다저녁도 빵인가 합체튀김소보로랑 생도넛 부추빵을 샀다들고 호텔로 토요코인 대전점여긴 대구점에 비해서 건물이 엄청 크네싱글룸이 없어서 디럭스룸으로 했다만원정도 더 비싼데 방이 완전 허허벌판 싱글룸은 좀 비좁았는데 넓어서 좋긴좋구만세탁기도 있고 조리대와 싱크대도 있다1인용 식기도 준비되어있음음료수 시원하게 먹으려고 1층가서 얼음을 잔뜩 받아왔다시내가서 닭강정을 사오려다가 바로앞에 맘스터치가 있길래 치킨을 사왔다맘스터치 찜햇닭강정살짝 매운데 깐풍기 느낌도 나고 맛있다배부르게 먹고 티비 보다가 잠듬']

In [57]:
for i in tqdm(range(len(data_df))):
    data_list = data_df['text'].iloc[i]
    data_df['text'].iloc[i] = list(map(dotAdd, data_list)) 

100%|██████████| 14/14 [00:00<00:00, 5108.77it/s]


In [58]:
for i in range(len(data_df)): # 5문장 정도만 내용 확인
    data_list = data_df['text'].iloc[i]
    print(f'{i}번째 데이터: {data_list}')
    if i == 5: 
        print('...')
        break

0번째 데이터: ['21년 4월 17일 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다 좀 벽이 있고 특별히 어려운 관계는 아니지만 그렇다고 일부러 연락해서 만나고 그러기엔 엄청 어색하다고우리 집안이 좀 이상한건지 대부분의 사촌동생들과 아주 친하게 지낸다.', '자주 연락하고 같이 식사 술 마시고 거리낌 없이 집에서 자고가기도 하고아', '참고로 아버지 집안의 형제가 7남매 어머니 집안은 8남매 대부분 2명의 자녀라 내 사촌들의 수는 엄청나다.', '크큽사회 초년생의 사촌들은 아무래도 내가 삼촌같아서 잘 어울리긴 힘들고나이가 엇 비슷한 사촌들과 좀 더 친하긴 하다.', '그 사촌 동생들 중 이라는 사촌이 있는데 오랜기간 공부하고 연구해서 박사가 되더니최근까지 서울에 대학에서 연구를 하다가 올해 대전에 있는 연구소로 취업을 하였다.', '회사에 있는동안 오피스텔이나 기숙사에서 지낼걸 생각했더니 이래저래 해서 대전 둔산동에 집을 매입했고 내부 인테리어 마쳤다고 했다.', '집들이도 할겸 동생 취업도 축하도 할겸마침 날씨는 좋은데 주말에 몸이 근질근질 하기도 했고1년정도의 장기간 코로나로 집에 더 뒹굴면 진짜 우울증 심하게 올것 같은 느낌이였다.', '그렇게 토요일 오후 차를 끌고 대전 둔산동 사촌집으로 갔다.', '야탑에 있는 회사에서 출발하였는데 2시간 좀 넘게 걸렸다고속도로는 막히지 않았는데 혼자가려니 엄청 지루했다.', '둔산동 동생집 주차장에 차를 세우고 간단히 집 구경을 하고 대부분 만족스러웠지만정리 빈약한 부분 오지랍 하면서 갈구기도 하고 식사를 하러 나왔다.', '출발 전부터 오늘은 이상하게 양꼬치가 먹고 싶었다.', '지글지글한 양꼬치에 맥주가 너무나 땡기던날사실 난 대전과 연관되는게 없어 주기적인 방문은 없다.', '주말에 필 받아서 친구랑 성심당에서 빵먹고 오기 바람쐬고 싶으니 고속도로 운전해서 가다가 얼떨결에 대전에서 치킨먹고 올라오기 토요일 오후 친구가 급하게 호출해서 뭔일인가하며 친구 차를 탔더니 대전에 거래에 혼자가기 심심

In [59]:
for i in tqdm(range(len(data_df))):
    data_list = data_df['text'].iloc[i]
    data_df['text'].iloc[i] = ' '.join(data_list)

100%|██████████| 14/14 [00:00<00:00, 6133.30it/s]


In [60]:
data_df['text'].iloc[0]

'21년 4월 17일 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다 좀 벽이 있고 특별히 어려운 관계는 아니지만 그렇다고 일부러 연락해서 만나고 그러기엔 엄청 어색하다고우리 집안이 좀 이상한건지 대부분의 사촌동생들과 아주 친하게 지낸다. 자주 연락하고 같이 식사 술 마시고 거리낌 없이 집에서 자고가기도 하고아 참고로 아버지 집안의 형제가 7남매 어머니 집안은 8남매 대부분 2명의 자녀라 내 사촌들의 수는 엄청나다. 크큽사회 초년생의 사촌들은 아무래도 내가 삼촌같아서 잘 어울리긴 힘들고나이가 엇 비슷한 사촌들과 좀 더 친하긴 하다. 그 사촌 동생들 중 이라는 사촌이 있는데 오랜기간 공부하고 연구해서 박사가 되더니최근까지 서울에 대학에서 연구를 하다가 올해 대전에 있는 연구소로 취업을 하였다. 회사에 있는동안 오피스텔이나 기숙사에서 지낼걸 생각했더니 이래저래 해서 대전 둔산동에 집을 매입했고 내부 인테리어 마쳤다고 했다. 집들이도 할겸 동생 취업도 축하도 할겸마침 날씨는 좋은데 주말에 몸이 근질근질 하기도 했고1년정도의 장기간 코로나로 집에 더 뒹굴면 진짜 우울증 심하게 올것 같은 느낌이였다. 그렇게 토요일 오후 차를 끌고 대전 둔산동 사촌집으로 갔다. 야탑에 있는 회사에서 출발하였는데 2시간 좀 넘게 걸렸다고속도로는 막히지 않았는데 혼자가려니 엄청 지루했다. 둔산동 동생집 주차장에 차를 세우고 간단히 집 구경을 하고 대부분 만족스러웠지만정리 빈약한 부분 오지랍 하면서 갈구기도 하고 식사를 하러 나왔다. 출발 전부터 오늘은 이상하게 양꼬치가 먹고 싶었다. 지글지글한 양꼬치에 맥주가 너무나 땡기던날사실 난 대전과 연관되는게 없어 주기적인 방문은 없다. 주말에 필 받아서 친구랑 성심당에서 빵먹고 오기 바람쐬고 싶으니 고속도로 운전해서 가다가 얼떨결에 대전에서 치킨먹고 올라오기 토요일 오후 친구가 급하게 호출해서 뭔일인가하며 친구 차를 탔더니 대전에 거래에 혼자가기 심심하다며 납치당하기 야구 한참 빠져서 원정응원가기이렇게 방문한게 전부였다. 그래서 대

다시 문장 분리 수행

In [61]:
data_df['text'] = data_df['text'].map(kss.split_sentences)

문장분리 > 정규표현식 > 맞춤법 검사 > 다시 문단 > 문장분리를 수행했을때 이상하게 문장 중간에 짤리는 경우가 개선된 것을 확인할 수 있습니다. 마지막으로 문장 분리 후 다시 맞춤법 검사를 거치면 정제된 문장이 추출되는 것을 확인할 수 있습니다.

In [62]:
data_df['text'].iloc[0]

['21년 4월 17일 대전여행다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다 좀 벽이 있고 특별히 어려운 관계는 아니지만 그렇다고 일부러 연락해서 만나고 그러기엔 엄청 어색하다고우리 집안이 좀 이상한건지 대부분의 사촌동생들과 아주 친하게 지낸다.',
 '자주 연락하고 같이 식사 술 마시고 거리낌 없이 집에서 자고가기도 하고아 참고로 아버지 집안의 형제가 7남매 어머니 집안은 8남매 대부분 2명의 자녀라 내 사촌들의 수는 엄청나다.',
 '크큽사회 초년생의 사촌들은 아무래도 내가 삼촌같아서 잘 어울리긴 힘들고나이가 엇 비슷한 사촌들과 좀 더 친하긴 하다.',
 '그 사촌 동생들 중 이라는 사촌이 있는데 오랜기간 공부하고 연구해서 박사가 되더니최근까지 서울에 대학에서 연구를 하다가 올해 대전에 있는 연구소로 취업을 하였다.',
 '회사에 있는동안 오피스텔이나 기숙사에서 지낼걸 생각했더니 이래저래 해서 대전 둔산동에 집을 매입했고 내부 인테리어 마쳤다고 했다.',
 '집들이도 할겸 동생 취업도 축하도 할겸마침 날씨는 좋은데 주말에 몸이 근질근질 하기도 했고1년정도의 장기간 코로나로 집에 더 뒹굴면 진짜 우울증 심하게 올것 같은 느낌이였다.',
 '그렇게 토요일 오후 차를 끌고 대전 둔산동 사촌집으로 갔다.',
 '야탑에 있는 회사에서 출발하였는데 2시간 좀 넘게 걸렸다고속도로는 막히지 않았는데 혼자가려니 엄청 지루했다.',
 '둔산동 동생집 주차장에 차를 세우고 간단히 집 구경을 하고 대부분 만족스러웠지만정리 빈약한 부분 오지랍 하면서 갈구기도 하고 식사를 하러 나왔다.',
 '출발 전부터 오늘은 이상하게 양꼬치가 먹고 싶었다.',
 '지글지글한 양꼬치에 맥주가 너무나 땡기던날사실 난 대전과 연관되는게 없어 주기적인 방문은 없다.',
 '주말에 필 받아서 친구랑 성심당에서 빵먹고 오기 바람쐬고 싶으니 고속도로 운전해서 가다가 얼떨결에 대전에서 치킨먹고 올라오기 토요일 오후 친구가 급하게 호출해서 뭔일인가하며 친구 차를 탔더니 대전에 거래에 혼자가기 심심하

## 맞춤법 검사 후 문장 분리
* [py-hanspell](https://github.com/ssut/py-hanspell): 네이버 맞춤법 검사기를 기반으로 합니다, pip 설치시 error가 있어서 git clone을 활용해야합니다. 
* [부산대 맞춤법 검사기 using selenium](https://hong-yp-ml-records.tistory.com/99): 부산대 맞춤법 검사기는 python 2.x 버전에 해당해서, 해당 블로그에서 Selenium을 활용해 맞춤법 검사를 진행한다. 해당 블로그 실험에서는 py-hanspell의 성능이 나았으므로 맞춤법 검사는 py-hanspell로만 진행하겠습니다. 
* [py-hangul-checker](https://pypi.org/project/py-hangul-checker/)

### pykospacing 띄어쓰기 교정

* [띄어쓰기 교정 관련 블로그](https://jeongukjae.github.io/posts/korean-spacing-model/)
* [kospacing](https://github.com/haven-jeon/PyKoSpacing)
* [soyspacing](https://taeguu.tistory.com/55)

In [63]:
from pykospacing import Spacing # 띄어쓰기 교정 라이브러리 

In [64]:
spacing = Spacing(rules=['하지만']) # rule을 추가해줄 수 있다. 

In [65]:
for i in tqdm(range(len(data_df))):
    data_list = data_df['text'].iloc[i]
    data_df['text'].iloc[i] = list(map(spacing, data_list))

100%|██████████| 14/14 [00:17<00:00,  1.25s/it]


In [66]:
# data_df['text'].iloc[0]

### soyspacing 띄어쓰기 교정

In [67]:
!pwd

/home/aiffel-dj19/jungcheck/DataPreprocessing


In [68]:
from soyspacing.countbase import CountSpace

In [69]:
############ 모델 train
corpus_fname = os.path.join(os.getcwd(),'134963_norm.txt')
model = CountSpace() 

# model.train(corpus_fname) #학습 
# model_fname = './soyspacing.model' #모델 저장위치
# model.save_model(model_fname, json_format=False) #모델 저장

In [70]:
model.load_model('./soyspacing.model', json_format=False) #저장 후 모델 불러올때 사용

In [71]:
verbose=False

mc = 10  # min_count
ft = 0.3 # force_abs_threshold
nt =-0.3 # nonspace_threshold
st = 0.3 # space_threshold

In [72]:
from soyspacing.countbase import RuleDict

rule_fname = os.path.join(os.getcwd(),'space_rule.txt')
rule_dict = RuleDict(rule_fname)

In [73]:
for i in tqdm(range(len(data_df))):
    data_list = data_df['text'].iloc[i]
    result_list = []
    for sen in data_list:
        result, _ = model.correct(doc=sen,
                                  verbose=verbose,
                                  force_abs_threshold=ft,
                                  nonspace_threshold=nt,
                                  space_threshold=st,
                                  min_count=mc,
                                  rules=rule_dict)
        result_list.append(result)
    data_df['text'].iloc[i] = result_list

100%|██████████| 14/14 [00:00<00:00, 34.23it/s]


In [74]:
data_df['text'].iloc[0]

['21년 4월 17일 대전 여행 다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다 좀 벽이 있고 특별히 어려운 관계는 아니지만 그렇다고 일부러 연락해서 만나고 그러기엔 엄청 어색하다고 우리 집안이 좀 이상한 건지 대부분의 사촌 동생들과 아주 친하게 지낸다.',
 '자주 연락하고 같이 식사 술 마시고 거리낌 없이 집에서 자고가기도 하고 아 참고로 아버지 집안의 형제가 7남매 어머니 집안은 8남매 대부분 2명의 자녀라 내 사촌들의 수는 엄청나다.',
 '크큽 사회 초년생의 사촌들은 아무래도 내가 삼촌 같아서 잘 어울리긴 힘들고 나이가 엇 비슷한 사촌들과 좀 더 친 하긴 하다.',
 '그 사촌 동생들 중 이라는 사촌이 있는데 오랜 기간 공부하고 연구해서 박사가 되더니 최근까지 서울에 대학에서 연구를 하다가 올해 대전에 있는 연구소로 취업을 하였다.',
 '회사에 있는 동안 오피스텔이나 기숙사에서 지 낼 걸 생각했더니 이래저래 해서 대전 둔산동에 집을 매입했고 내부 인테리어 마쳤다고 했다.',
 '집들이도 할 겸 동생 취업도 축하도 할 겸 마침 날씨는 좋은데 주말에 몸이 근질 근질 하기도 했고 1년 정도의 장기간 코로나로 집에 더 뒹굴면 진짜 우울증 심하게 올 것 같은 느낌이였다.',
 '그렇게 토요일 오후 차를 끌고 대전 둔산동 사촌 집으로 갔다.',
 '야탑에 있는 회사에서 출발하였는데 2시간 좀 넘게 걸렸다고 속도로는 막히지 않았는데 혼자 가려니 엄청 지루했다.',
 '둔산동 동생 집 주차장에 차를 세우고 간단히 집 구경을 하고 대부분 만족스러웠지만 정리 빈약한 부분 오지랍 하면서 갈구기도 하고 식사를 하러 나왔다.',
 '출발 전부터 오늘은 이 상하게 양꼬치가 먹고 싶었다.',
 '지글지글한 양꼬치에 맥주가 너무나 땡기던 날 사실 난 대전과 연관되는 게 없어 주기적인 방문은 없다.',
 '주말에 필 받아서 친구랑 성심당에서 빵 먹고 오기 바람쐬고 싶으니 고속도로 운전해서 가 다가 얼떨결에 대전에서 치킨 먹고 올라오기 토요일 오후 친구가 급하게 호출해서

### py-hangul-check를 활용한 맞춤법 검사

In [75]:
from hangul_checker import KoreanSpellChecker

In [76]:
ksc = KoreanSpellChecker()

In [77]:
# for i in tqdm(range(len(data_df))):
#     try:
#         data_list = data_df['text'].iloc[i]
#         data_df['text'].iloc[i] = list(map(ksc.check_spelling, data_list))
#         if i <= 5:
#             print(f'맞춤법 교정전 {i}번째 데이터: {data_list[i]}')
#             print(f'맞춤법 교정후 {i}번째 데이터: {data_df['text'].iloc[i]}')
#     except:
#         print('Server Error')
#         break

API ERROR

### hanspell을 활용한 맞춤법 검사

In [78]:
from hanspell import spell_checker

In [79]:
data_df.head()

,date,description,text,name,search,keyword
1,210427,- 주말에 필 받아서 친구랑 성심당에서 빵먹고 오기. - 바람쐬고 싶으니 고속도로 ...,[21년 4월 17일 대전 여행 다른 친구들 이야기 들어보면 사촌들과의 관계는 친구...,그 녀석의 여행 이야기,성심당 여행 주말에,성심당
0,191018,이제 주말에 여유 생길 테니 자주 볼 수 있으면 좋겠다.^^ 2019대전 : 20개...,[2019 대전 20개월 아기와 함께 당일치기 여행 첫 차를 산 동생에게 친절한 형...,봄날,성심당 관광 주말에,성심당
1,191012,- 2019년 10월 12일 토요일 주말에 신랑이 같이 일하는 형님들과 배낚시를 간...,[2019년 10월 12일 토요일 주말에 신랑이 같이 일하는 형님들과 배낚시를 간다...,; ] 양 뚱의 BLOG,성심당 관광 주말에,성심당
2,190923,음식을 성심당 것 차 이들이 경기권으로 제가 주말에 들어가서 많아 다 법령도 열심히...,"[안녕하세요날이 갈수록 서늘 해지는 가을로 접어들었습니다., 활동하기 알맞은 날씨와...",부산현수막,성심당 관광 주말에,성심당
3,190923,주말에 태풍소식이 ㅠㅠ 이전 태풍때는 바람만 불기도 했었고 그 전 태풍때 서울엔 흐...,[대전 온천여행이 되버린 주말 유성 호텔 수영장 대욕장에서 놀고 왔어요 저희 가족은...,서남매와 제주살이중_♥,성심당 관광 주말에,성심당


In [80]:
# data_list = []
# result_list = []

# for i in tqdm(range(len(data_df))):
#     data_list = data_df['text'].iloc[i]
#     data_list = list(map(spell_checker.check, data_list))
# #     for j in range(len(data_list)):
# #         result = data_list[j].as_dict()['check']
# #         result_list.append(result)
# #     data_df['text'].iloc[i] = result_list
# #     if i <= 5:
# #         print(f'맞춤법 교정후 {i}번째 데이터: {result_list[i]}')

In [81]:
# test_list = data_df['text'].iloc[0]
# for i in tqdm(range(len(test_list))): 
#     result = spell_checker.check(test_list[i]) 
#     test_list = result.as_dict()['check']

In [82]:
# for i in tqdm(range(len(test_list))): 
#     try: 
#         result = spell_checker.check(test_list[i]) 
#         test_list[i] = result.as_dict()['check'] 
#     except: 
#         pass 

In [83]:
# for i in tqdm(range(len(test_list))): 
#     if test_list[i]['original'] != test_list[i]['checked']:
#         print(test_list[i]['original'])
#         print(test_list[i]['checked'])

In [84]:
# test_list = [test_list[i]['checked'] for i in range(len(test_list))]

In [85]:
spell_checker.check('시도햇보다 ')

Checked(result=True, original='시도햇보다 ', checked='시도해보다', errors=1, words=OrderedDict([('시도해보다', 4)]), time=2.958942174911499)

In [86]:
for i in tqdm(range(len(data_df))):
    data_list = data_df['text'].iloc[i]
    result_list = []
    for sen in data_list:
        result = spell_checker.check(sen)
        result = result.as_dict()['checked']
        result_list.append(result)
    data_df['text'].iloc[i] = result_list

100%|██████████| 14/14 [01:09<00:00,  4.97s/it]


In [87]:
data_df['text'].iloc[0]

['21년 4월 17일 대전 여행 다른 친구들 이야기 들어보면 사촌들과의 관계는 친구들보다 좀 벽이 있고 특별히 어려운 관계는 아니지만 그렇다고 일부러 연락해서 만나고 그러기엔 엄청 어색하다고 우리 집안이 좀 이상한 건지 대부분의 사촌 동생들과 아주 친하게 지낸다.',
 '자주 연락하고 같이 식사 술 마시고 거리낌 없이 집에서 자고 가기도 하고 아 참고로 아버지 집안의 형제가 7남매 어머니 집안은 8남매 대부분 2명의 자녀라 내 사촌들의 수는 엄청나다.',
 '크큽 사회 초년생의 사촌들은 아무래도 내가 삼촌 같아서 잘 어울리긴 힘들고 나이가 엇 비슷한 사촌들과 좀 더 친 하긴 하다.',
 '그 사촌 동생들 중이라는 사촌이 있는데 오랜 기간 공부하고 연구해서 박사가 되더니 최근까지 서울에 대학에서 연구를 하다가 올해 대전에 있는 연구소로 취업을 하였다.',
 '회사에 있는 동안 오피스텔이나 기숙사에서 지 낼 걸 생각했더니 이래저래 해서 대전 둔산동에 집을 매입했고 내부 인테리어 마쳤다고 했다.',
 '집들이도 할 겸 동생 취업도 축하도 할 겸 마침 날씨는 좋은데 주말에 몸이 근질 근질 하기도 했고 1년 정도의 장기간 코로나로 집에 더 뒹굴면 진짜 우울증 심하게 올 것 같은 느낌이었다.',
 '그렇게 토요일 오후 차를 끌고 대전 둔산동 사촌 집으로 갔다.',
 '야탑에 있는 회사에서 출발하였는데 2시간 좀 넘게 걸렸다고 속도로는 막히지 않았는데 혼자 가려니 엄청 지루했다.',
 '둔산동 동생 집 주차장에 차를 세우고 간단히 집 구경을 하고 대부분 만족스러웠지만 정리 빈약한 부분 오지랖 하면서 갈구기도 하고 식사를 하러 나왔다.',
 '출발 전부터 오늘은 이 상하게 양 꼬치가 먹고 싶었다.',
 '지글지글한 양 꼬치에 맥주가 너무나 당기던 날 사실 난 대전과 연관되는 게 없어 주기적인 방문은 없다.',
 '주말에 필받아서 친구랑 성심당에서 빵 먹고 오기 바람 쐬고 싶으니 고속도로 운전해서 가 다가 얼떨결에 대전에서 치킨 먹고 올라오기 토요일 오후 친구가 급하게 호출

## 정제 후 csv로 저장

In [88]:
data_df['keyword'] = keyword_en
data_df.head()

,date,description,text,name,search,keyword
1,210427,- 주말에 필 받아서 친구랑 성심당에서 빵먹고 오기. - 바람쐬고 싶으니 고속도로 ...,[21년 4월 17일 대전 여행 다른 친구들 이야기 들어보면 사촌들과의 관계는 친구...,그 녀석의 여행 이야기,성심당 여행 주말에,sungsimdang_bakery
0,191018,이제 주말에 여유 생길 테니 자주 볼 수 있으면 좋겠다.^^ 2019대전 : 20개...,[2019 대전 20개월 아기와 함께 당일치기 여행 첫 차를 산 동생에게 친절한 형...,봄날,성심당 관광 주말에,sungsimdang_bakery
1,191012,- 2019년 10월 12일 토요일 주말에 신랑이 같이 일하는 형님들과 배낚시를 간...,[2019년 10월 12일 토요일 주말에 신랑이 같이 일하는 형님들과 배낚시를 간다...,; ] 양 뚱의 BLOG,성심당 관광 주말에,sungsimdang_bakery
2,190923,음식을 성심당 것 차 이들이 경기권으로 제가 주말에 들어가서 많아 다 법령도 열심히...,"[안녕하세요 날이 갈수록 서늘 해지는 가을로 접어들었습니다., 활동하기 알맞은 날씨...",부산현수막,성심당 관광 주말에,sungsimdang_bakery
3,190923,주말에 태풍소식이 ㅠㅠ 이전 태풍때는 바람만 불기도 했었고 그 전 태풍때 서울엔 흐...,[대전 온천여행이 돼버린 주말 유성 호텔 수영장 대욕장에서 놀고 왔어요 저희 가족은...,서남매와 제주살이중_♥,성심당 관광 주말에,sungsimdang_bakery


In [89]:
def saveCsv(root_path, key_en, df):
    print(f'저장 경로: {root_path}')
    df.to_csv(root_path+key_en+'_nv.csv', header=True, index=False)

In [90]:
save_path = os.path.join(os.getcwd(), 'filtered/naver_review/')
save_path

'/home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/naver_review/'

In [91]:
saveCsv(save_path, keyword_en, data_df)

저장 경로: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/naver_review/


In [92]:
!pwd
!ls /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/naver_review

/home/aiffel-dj19/jungcheck/DataPreprocessing
sungsimdang_bakery_nv.csv
